We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a 'db' folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"
print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

Using DB_PATH /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb


We verify that the primary panel aligns with the expected regular-season schedule and row counts across all seasons

In [2]:
required = ["pbp", "schedules", "team_week_panel"]
existing = set(con.execute("SHOW TABLES").df()["name"].tolist())

missing = [t for t in required if t not in existing]

print("Missing", missing)
print("OK" if not missing else "STOP, fix missing tables before continuing")

Missing []
OK


We inspect the play-by-play schema to record which optional columns exist, allowing subsequent logic to adapt safely without assuming column presence.

In [3]:
pbp_cols_df = con.execute("DESCRIBE pbp").df()
pbp_cols = set(pbp_cols_df["column_name"].tolist())

optional_flags = {
    "return_team": "return_team" in pbp_cols,
    "kickoff_type": "kickoff_type" in pbp_cols,
    "onside_kick": "onside_kick" in pbp_cols,
    "safety_kick": "safety_kick" in pbp_cols,
    "free_kick": "free_kick" in pbp_cols,
    "desc": "desc" in pbp_cols,
}

print("Optional columns present", optional_flags)

pbp_cols_df

Optional columns present {'return_team': True, 'kickoff_type': False, 'onside_kick': False, 'safety_kick': False, 'free_kick': False, 'desc': True}


,column_name,column_type,null,key,default,extra
0,play_id,DOUBLE,YES,None,None,None
1,game_id,VARCHAR,YES,None,None,None
2,old_game_id,VARCHAR,YES,None,None,None
3,home_team,VARCHAR,YES,None,None,None
4,away_team,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
367,xyac_median_yardage,INTEGER,YES,None,None,None
368,xyac_success,DOUBLE,YES,None,None,None
369,xyac_fd,DOUBLE,YES,None,None,None
370,xpass,DOUBLE,YES,None,None,None


We check that team identifiers in the panel align with those in the play-by-play data, ensuring join keys are consistent across datasets to prevent failed matches

In [4]:
panel_cols_df = con.execute("DESCRIBE team_week_panel").df()
panel_cols = set(panel_cols_df["column_name"].tolist())

if "team_id" in panel_cols:
    PANEL_TEAM_COL = "team_id"
elif "team" in panel_cols:
    PANEL_TEAM_COL = "team"
else:
    raise ValueError(f"Could not find a team column in team_week_panel, columns are {sorted(panel_cols)[:50]}")

print("Using panel team column", PANEL_TEAM_COL)

panel_teams = set(
    con.execute(f"SELECT DISTINCT {PANEL_TEAM_COL} AS team_key FROM team_week_panel WHERE {PANEL_TEAM_COL} IS NOT NULL")
      .df()["team_key"]
      .tolist()
)

pbp_posteams = set(
    con.execute("SELECT DISTINCT posteam FROM pbp WHERE posteam IS NOT NULL")
      .df()["posteam"]
      .tolist()
)

intersect = panel_teams.intersection(pbp_posteams)

print("Distinct panel teams", len(panel_teams))
print("Distinct posteam in pbp", len(pbp_posteams))
print("Intersection size", len(intersect))

print("Sample posteam not in panel", sorted(list(pbp_posteams - panel_teams))[:25])
print("Sample panel team not in pbp", sorted(list(panel_teams - pbp_posteams))[:25])


Using panel team column team
Distinct panel teams 35
Distinct posteam in pbp 32
Intersection size 32
Sample posteam not in panel []
Sample panel team not in pbp ['OAK', 'SD', 'STL']


Quick sanity check to verify that we are 

In [5]:
con.execute(f"""
SELECT
  {PANEL_TEAM_COL} AS team,
  MIN(season) AS min_season,
  MAX(season) AS max_season,
  COUNT(*) AS rows
FROM team_week_panel
WHERE {PANEL_TEAM_COL} IN ('OAK','SD','STL')
GROUP BY 1
ORDER BY 1
""").df()


,team,min_season,max_season,rows
0,OAK,2012,2019,128
1,SD,2012,2016,80
2,STL,2012,2015,64
